In [ ]:
pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify
import os

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Set the authentication token
ngrok.set_auth_token("2f0VPhZJtS98go0K8UjVxuYloKp_6wdLgZjzuKYrFWj2HiSPh")

# Set up a tunnel to the Flask-like server running on port 8888
public_url = ngrok.connect(port='8888')
print('Public URL:', public_url)

PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
!pip install google.colab
from google.colab.patches import cv2_imshow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00


In [ ]:
from collections import defaultdict
import math
import cv2
import numpy as np

from ultralytics import YOLO

In [ ]:
# Conversion of pixels to cms
def pixel_to_cm(pixel_value, pixels_per_cm):

    return pixel_value / pixels_per_cm
# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    val=pixel_to_cm(math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2),100)
    return val

def Suspicious_Activity_Detection(file_path):
  model = YOLO('yolov8n.pt')

  time_interval = 0.25  # Extract a frame every 0.25 seconds
  last_extract_time = 0


In [ ]:
#detection and tracking

app = Flask(__name__)

@app.route('/process', methods=['POST'])
uploaded_file = request.files['video']
if uploaded_file.filename != '':
    file_path = os.path.join('/content', uploaded_file.filename)
    uploaded_file.save(video_path)

# Open the video file
  video_path = file_path
  cap = cv2.VideoCapture(video_path)

  track_history = defaultdict(lambda: [])
  track_class = defaultdict(lambda: [])
# Getting all the names of the different classes
  names = model.names
  countA = 0
# Loop through the video frames
  while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        countA += 1
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if current_time - last_extract_time >= time_interval:
            last_extract_time = current_time

            results = model.track(frame, persist=True)

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            class_ids = results[0].boxes.cls.int().cpu().tolist()  # Retrieve class IDs directly
            for id,class_ in zip(track_ids,class_ids):
              track_class[id] = class_
            print(track_ids)
            print(class_ids)
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            print(countA)

            if(countA == 109):
              annotated_frame1 = annotated_frame
            if(countA == 81):
              annotated_frame1 = annotated_frame
            if(countA == 122):
              annotated_frame1 = annotated_frame
            if(countA == 128):
              annotated_frame1 = annotated_frame


            # Plot the tracks
            for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                x, y, w, h = box
                track = track_history[track_id]



                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            # The dictionaries have been created
            #print(track_class)
            #print(track_history)

            # Display the annotated frame
            #cv2_imshow(annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
    else:
        break

# Release the video capture object and close the display window
  cap.release()
  cv2.destroyAllWindows()

  print(track_class)
  person_ids = []
  car_ids = []
  for id in results[0].boxes.cls:
    print(id)
# Iterate through the boxes and IDs
  for (id_1,val1), (id_2,val2) in zip(track_class.items(),track_history.items()):
    cls = int(val1)  # Convert class to integer

    # Check the class and append the ID to the respective list
    if cls == 0:
        person_ids.append(int(id_2))
    elif cls == 2:
        car_ids.append(int(id_2))

# Print the lists
  print("IDs with class 0:", person_ids)
  print("IDs with class 2:", car_ids)



# Dictionary to store distances between persons and cars
  distances = defaultdict(dict)

# Iterate over each person and car pair
  for person_id in person_ids:
    for car_id in car_ids:
        if person_id != car_id:  # Ensure we're not comparing a person with themselves
            # Get the track history of the person and car
            person_track = track_history[person_id]
            car_track = track_history[car_id]
            if person_track and car_track:  # Check if both tracks are non-empty
                # Calculate distances between all points in the tracks
                distances[person_id][car_id] = [calculate_distance(person_point, car_point) for person_point in person_track for car_point in car_track]

# Print distances
  for person_id, car_distances in distances.items():
    for car_id, distance_list in car_distances.items():
        print(f"Distances between person {person_id} and car {car_id}: {distance_list}")


# Define threshold values
  loitering_distance_threshold = 1
  consecutive_frames_threshold = 150

# Dictionary to store loitering frames for each person-car pair
  loitering_frames = defaultdict(int)

# Iterate over the distances between persons and cars
  for person_id, car_distances in distances.items():
    for car_id, distance_list in car_distances.items():
        for distance in distance_list:
            if distance < loitering_distance_threshold:
                # Increment loitering frames count for the person-car pair
                loitering_frames[(person_id, car_id)] += 1
            else:
                # Reset loitering frames count if distance exceeds threshold
                #loitering_frames[(person_id, car_id)] = 0
                break
# Check for loitering based on consecutive frames threshold
  output_strings = []
  for (person_id, car_id), count in loitering_frames.items():
    if count >= consecutive_frames_threshold:
        output_strings.append(f"Person {person_id} is loitering near car {car_id}.")
    else:
        output_strings.append(f"No loitering detected for person {person_id} near car {car_id}.")

# Return the list of output strings
  output_image=annotated_frame1

In [ ]:
def return_video_result(video_path):
    return jsonify({'result': results})

In [ ]:
if __name__ == '__main__':
    app.run(port=8888)